# Observation / Action Specifiction (Clean Implementation)

TODO:
- [x] Observation Spec `replay_df`
   - Champs
   - Minions
   - Turrets
   - Monsters
   - Missiles
- [x] Action Spec `replay_df`:
   - Movement
   - Auto Attack
   - Spell
     - Q
     - W
     - E
     - D (Summoner1)
     - F (Summoner2)
   - Recall
   - No-Op?

## Populate Game Object Dataframes

In [73]:
import os
import pandas as pd
import numpy as np

db_replays_dir = "/Users/joe/Downloads/DB"
db_replays = os.listdir(db_replays_dir)

In [74]:
import sqlite3
import pandas as pd
import os
db_replay = os.path.join(db_replays_dir, db_replays[0])
con = sqlite3.connect(db_replay)

In [80]:
GAME_OBJECT_LIST = ["champs", "turrets", "minions", "missiles", "monsters"]

In [104]:
MAX_OBJS = [10, 30, 30, 30, 30]

In [105]:
df_s = {
    obj:pd.read_sql(f"SELECT * FROM {obj};", con) for obj in GAME_OBJECT_LIST}

## Dataframe Preprocessing

### Data Cleaning for all Game Object Dataframes

In [106]:
for obj in GAME_OBJECT_LIST:
    df_s[obj] = df_s[obj].drop_duplicates(
        subset=["time", "name"])
    df_s[obj] = df_s[obj][
        df_s[obj]["time"] > 15]
    df_s[obj] = df_s[obj].drop("game_id", axis=1)

### Data Normalisation for Champs DF

In [107]:
df_s["champs"].loc[df_s["champs"]['q_cd'] < 0, 'q_cd'] = 0
df_s["champs"].loc[df_s["champs"]['w_cd'] < 0, 'w_cd'] = 0
df_s["champs"].loc[df_s["champs"]['e_cd'] < 0, 'e_cd'] = 0
df_s["champs"].loc[df_s["champs"]['r_cd'] < 0, 'r_cd'] = 0
df_s["champs"].loc[df_s["champs"]['d_cd'] < 0, 'd_cd'] = 0
df_s["champs"].loc[df_s["champs"]['f_cd'] < 0, 'f_cd'] = 0

## Dataframe Feature Engineering

In [108]:
player_df = df_s["champs"][df_s["champs"]["name"] == "Ezreal"]

In [109]:
player_df

,time,name,hp,max_hp,mana,max_mana,armor,mr,ad,ap,...,w_name,w_cd,e_name,e_cd,r_name,r_cd,d_name,d_cd,f_name,f_cd
503,15.034752,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.965248,SummonerFlash,0.965248
513,15.303469,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.696531,SummonerFlash,0.696531
523,15.581064,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.418936,SummonerFlash,0.418936
533,15.849633,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.150367,SummonerFlash,0.150367
543,16.139160,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6643,178.674740,Ezreal,850.45,850.45,327.76978,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,13.740845,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
6653,178.970980,Ezreal,850.45,850.45,328.76727,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,13.444611,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
6663,179.318790,Ezreal,850.45,850.45,328.76727,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,13.096802,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
6683,179.636140,Ezreal,850.45,850.45,329.76477,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,12.779449,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000


## Observation Spec

### Init `replay_df`

In [110]:
replay_df = pd.DataFrame()
times = df_s["champs"].drop_duplicates(subset=['time'])["time"]
replay_df["time"] = times

### Add Distance Between Local Player and All Game Objects

In [111]:
player_df = df_s["champs"][df_s["champs"]["name"] == "Ezreal"]

In [112]:
def add_distances(original_df):
    # Step 1: Filter out Player's data
    player_df_data = player_df[['time', 'pos_x', 'pos_z']]

    # Step 2: Merge with the original DataFrame on 'time'
    merged_df = original_df.merge(player_df_data, on='time', suffixes=('', '_player'))

    # Step 3: Calculate Euclidean distance
    merged_df['distance_from_player_x'] = abs(
        merged_df["pos_x_player"] - merged_df["pos_x"])
    merged_df['distance_from_player_z'] = abs(
        merged_df["pos_z_player"] - merged_df["pos_z"])
    merged_df['distance_from_player'] = np.sqrt(
        (merged_df['pos_x'] - merged_df['pos_x_player'])**2 +
        (merged_df['pos_z'] - merged_df['pos_z_player'])**2)
    merged_df = merged_df.drop(columns=["pos_x_player", "pos_z_player"])

    # Filter out the rows where the champion is Ezreal, as we don't need the distance of Ezreal to himself
    return merged_df

for obj in GAME_OBJECT_LIST:
    df_s[obj] = add_distances(df_s[obj])

### Flatten Observations

In [113]:
testdf_s = {}

#### Flatten Each Dataframe

In [139]:
times = df_s["champs"]["time"].unique()

def flatten_obs(df, max_objs, times):
    print(df.columns)
    new_cols = [f"{c}_{idx}"
                for idx in range(max_objs)
                for c in df.columns[1:]]
    new_cols = ["time"] + new_cols
    obs   = []
    for tm in times:
        cur = df[df["time"] == tm]
        vals = cur.values[:, 1:]
        vals = vals[:max_objs, :]

        if cur.shape[0] < max_objs:
            padding_val = max_objs - vals.shape[0]
            padding = np.zeros((padding_val, vals.shape[1]))
            vals = np.vstack((vals, padding))
    
        # Flatten
        new_vals = np.hstack(vals)

        # Append
        obs.append(new_vals)

    # Combine
    obs   = np.vstack(obs)
    times = np.expand_dims(np.array(times), axis=1)
    obs   = np.hstack((times, obs))
    print(obs.shape)
    obs_df = pd.DataFrame(data=obs, columns=new_cols)

    return obs_df

for obj, max_objs in zip(GAME_OBJECT_LIST, MAX_OBJS):
    print("OBJ, MAX:", obj, max_objs)
    testdf_s[obj] = flatten_obs(df_s[obj], max_objs, times)

OBJ, MAX: champs 10
Index(['time', 'name', 'hp', 'max_hp', 'mana', 'max_mana', 'armor', 'mr', 'ad',
       'ap', 'level', 'atk_range', 'visible', 'team', 'pos_x', 'pos_z',
       'q_name', 'q_cd', 'w_name', 'w_cd', 'e_name', 'e_cd', 'r_name', 'r_cd',
       'd_name', 'd_cd', 'f_name', 'f_cd', 'distance_from_player_x',
       'distance_from_player_z', 'distance_from_player'],
      dtype='object')
(574, 301)
OBJ, MAX: turrets 30
Index(['time', 'name', 'hp', 'max_hp', 'mana', 'max_mana', 'armor', 'mr', 'ad',
       'ap', 'level', 'atk_range', 'visible', 'team', 'pos_x', 'pos_z',
       'distance_from_player_x', 'distance_from_player_z',
       'distance_from_player'],
      dtype='object')
(574, 541)
OBJ, MAX: minions 30
Index(['time', 'name', 'hp', 'max_hp', 'mana', 'max_mana', 'armor', 'mr', 'ad',
       'ap', 'level', 'atk_range', 'visible', 'team', 'pos_x', 'pos_z',
       'distance_from_player_x', 'distance_from_player_z',
       'distance_from_player'],
      dtype='object')
(574, 

#### Combine All Flattened Dataframes

In [165]:
times_unsqueeze = np.expand_dims(times, 1)
replay_df_vals = [df.iloc[:, 1:] for df in testdf_s.values()]
replay_df_vals = np.hstack(replay_df_vals)
print(times_unsqueeze.shape, replay_df_vals.shape)
replay_df_vals = np.hstack((times_unsqueeze, replay_df_vals))

(574, 1) (574, 2340)


In [166]:
replay_df_vals.shape

(574, 2341)

In [167]:
def flatten_list(lst):
    return [x for xs in lst for x in xs]

replay_df_cols = [list(testdf_s[k].columns[1:].values) for k in testdf_s.keys()]
replay_df_cols = flatten_list(replay_df_cols)
replay_df_cols = ["time"] + replay_df_cols

In [168]:
replay_df = pd.DataFrame(
    data=replay_df_vals,
    columns=replay_df_cols)
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,ap_29,level_29,atk_range_29,visible_29,team_29,pos_x_29,pos_z_29,distance_from_player_x_29,distance_from_player_z_29,distance_from_player_29
0,15.034752,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.303469,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15.581064,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15.849633,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16.13916,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.67474,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
570,178.97098,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
571,179.31879,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
572,179.63614,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
